In [4]:
# Import the required libraries and dependencies
# Import Modules
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
import datetime
from pandas_datareader import data as pdr
import yfinance as yf
from pandas.tseries.offsets import DateOffset

In [5]:
# map ticker as object
# ES
ticker = yf.Ticker("QQQ")

In [6]:
# specify time series start / end dates in "yyyy-mm-dd" format
ticker_df = ticker.history(start="2012-01-01", end="2021-12-31")


display(ticker_df.head())
display(ticker_df.tail())

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-01-03,51.671302,51.925526,51.526030,51.662224,39514100,0.0,0
2012-01-04,51.580510,51.952768,51.353523,51.880131,29403400,0.0,0
2012-01-05,51.771184,52.352270,51.571434,52.306873,41260600,0.0,0
2012-01-06,52.315956,52.606499,52.116210,52.488468,46325200,0.0,0
2012-01-09,52.624643,52.642803,52.143432,52.315941,39195500,0.0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-12-23,394.309998,398.429993,394.140015,396.920013,29595500,0.0,0
2021-12-27,398.250000,403.480011,398.220001,403.480011,32820700,0.0,0
2021-12-28,404.399994,404.579987,400.709991,401.609985,33113700,0.0,0
2021-12-29,401.750000,403.049988,399.109985,401.549988,32248500,0.0,0
2021-12-30,401.329987,403.570007,399.750000,400.350006,23850100,0.0,0


In [9]:
# Filter the date index and close columns
signals_df = ticker_df.loc[:,["Close"]]

# Set the short window and long windows
short_window = 7
long_window = 25

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA_fast'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA_slow'] = signals_df['Close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA_fast'][short_window:] > signals_df['SMA_slow'][short_window:], 1.0, 0.0
)

# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
signals_df.tail(10)

,Close,SMA_fast,SMA_slow,Signal,Entry/Exit
Date,,,,,
2021-12-16,386.346558,393.169272,393.930066,0.0,-1.0
2021-12-17,384.419006,391.071952,393.703156,0.0,0.0
2021-12-20,380.690002,389.280840,393.162894,0.0,0.0
2021-12-21,389.209991,388.096222,392.967029,0.0,0.0
2021-12-22,393.950012,388.409132,392.848507,0.0,0.0
2021-12-23,396.920013,389.725586,392.840396,0.0,0.0
2021-12-27,403.480011,390.716513,392.930095,0.0,0.0
2021-12-28,401.609985,392.897003,392.855508,1.0,1.0
2021-12-29,401.549988,395.344286,392.965881,1.0,0.0


In [107]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='orange',
    marker='v',
    size=100,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=100,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA_fast', 'SMA_slow']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="QQQ - SMA Fast, SMA Slow, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [31]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Buy a 500 share position when the dual moving average crossover Signal equals 1 (SMA50 is greater than SMA100)
# Sell a 500 share position when the dual moving average crossover Signal equals 0 (SMA50 is less than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Position']

# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the portfolio cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(150)

,Close,SMA_fast,SMA_slow,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2021-05-28,332.748718,330.577855,329.439662,1.0,1.0,500.0,500.0,166374.359131,27136.848450,193511.207581,0.000000,0.935112
2021-06-01,331.642639,331.145839,329.088508,1.0,0.0,500.0,0.0,165821.319580,27136.848450,192958.168030,-0.002858,0.929582
2021-06-02,332.290344,332.065430,328.822255,1.0,0.0,500.0,0.0,166145.172119,27136.848450,193282.020569,0.001678,0.932820
2021-06-03,328.832642,331.708126,328.463529,1.0,0.0,500.0,0.0,164416.320801,27136.848450,191553.169250,-0.008945,0.915532
2021-06-04,334.412811,332.082511,328.279382,1.0,0.0,500.0,0.0,167206.405640,27136.848450,194343.254089,0.014566,0.943433
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,396.920013,389.725586,392.840396,0.0,0.0,0.0,0.0,0.000000,212090.614319,212090.614319,0.000000,1.120906
2021-12-27,403.480011,390.716513,392.930095,0.0,0.0,0.0,0.0,0.000000,212090.614319,212090.614319,0.000000,1.120906
2021-12-28,401.609985,392.897003,392.855508,1.0,1.0,500.0,500.0,200804.992676,11285.621643,212090.614319,0.000000,1.120906


In [35]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='black',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="QQQ Trading Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [36]:
# Create a list for the column name
columns = ['Backtest']

# Create a list holding the names of the new evaluation metrics
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [37]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annualized Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.078269
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [38]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df

,Backtest
Annualized Return,0.078269
Cumulative Returns,1.114606
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [39]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.078269
Cumulative Returns,1.114606
Annual Volatility,0.079791
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [40]:
# Calculate Sharpe ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.078269
Cumulative Returns,1.114606
Annual Volatility,0.079791
Sharpe Ratio,0.980929
Sortino Ratio,NaN


In [41]:
# Calculate downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0,
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Review the DataFrame
sortino_ratio_df.tail()

,Portfolio Daily Returns,Downside Returns
Date,,
2021-12-23,0.000000,0.000000e+00
2021-12-27,0.000000,0.000000e+00
2021-12-28,0.000000,0.000000e+00
2021-12-29,-0.000141,2.000618e-08
2021-12-30,-0.002829,8.005145e-06


In [42]:
# Calculate the Sortino ratio

# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
)

# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annualized Return,0.078269
Cumulative Returns,1.114606
Annual Volatility,0.079791
Sharpe Ratio,0.980929
Sortino Ratio,1.358444


In [43]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Holding',
        'Exit Portfolio Holding',
        'Profit/Loss']
)


In [44]:
# Initialize iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

In [46]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'QQQ',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,QQQ,2012-02-07,2012-04-13,500.0,56.410797,60.199085,28205.398560,30099.542618,1894.144058
1,QQQ,2012-05-03,2012-05-04,500.0,60.344589,58.843937,30172.294617,29421.968460,-750.326157
2,QQQ,2012-06-14,2012-07-17,500.0,56.715733,57.912632,28357.866287,28956.315994,598.449707
3,QQQ,2012-07-19,2012-10-02,500.0,59.343803,62.776646,29671.901703,31388.322830,1716.421127
4,QQQ,2012-11-28,2012-12-28,500.0,59.897015,58.631081,29948.507309,29315.540314,-632.966995
5,QQQ,2013-01-03,2013-02-25,500.0,61.453243,60.956821,30726.621628,30478.410721,-248.210907
6,QQQ,2013-03-06,2013-04-09,500.0,62.979225,63.281628,31489.612579,31640.813828,151.201248
7,QQQ,2013-04-10,2013-04-22,500.0,64.516296,63.382984,32258.148193,31691.492081,-566.656113
8,QQQ,2013-04-26,2013-06-06,500.0,64.101677,66.709221,32050.838470,33354.610443,1303.771973
9,QQQ,2013-07-09,2013-08-22,500.0,67.558029,70.395683,33779.014587,35197.841644,1418.827057


In [106]:
# Save backtested evaluation metrics as csv for use in plot metrics notebook 
# Specifify columns in header
header = ['Entry Date', 'Exit Date', 'Entry Share Price', 'Exit Share Price','Entry Portfolio Holding','Exit Portfolio Holding','Profit/Loss']
trade_evaluation_df.to_csv('metrics.csv', columns = header)